In [1]:
import statistics
import matplotlib.pyplot as plt
import pymysql
import config
import transformations
import ml
from sklearn.model_selection import KFold
import pandas as pd

C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery", conn)

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["MagicHourType"] = pd.Categorical(RideWaits["MagicHourType"])
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days


In [3]:
originalData = RideWaits

,RideId,Date,Time,Wait,Name,OpeningDate,Tier,Location,IntellectualProp,Status,...,Weekend,CharacterExperience,inEMH,validTime,EMHDay,TimeSinceOpen,TimeSinceMidday,MagicHourType,MinutesSinceOpen,TimeSinceRideOpen
0,0,2018-05-07,12:30:00,35,Astro Orbiter,1995-02-25,minor_attraction,Tomorrowland,None,1,...,0,0,0,1,1,3,2,Night,210.0,8472
1,2,2018-05-07,12:30:00,45,Big Thunder Mountain Railroad,1980-09-23,headliner,Frontierland,None,1,...,0,0,0,1,1,3,2,Night,210.0,13740
2,3,2018-05-07,12:30:00,45,Buzz Lightyears Space Ranger Spin,1998-10-07,minor_attraction,Tomorrowland,Pixar,1,...,0,0,0,1,1,3,2,Night,210.0,7152
3,6,2018-05-07,12:30:00,40,Dumbo the Flying Elephant,1971-10-01,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,17020
4,7,2018-05-07,12:30:00,25,Enchanted Tales with Belle,2012-12-06,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,1978


In [4]:
keyFeatures = ["Name","MagicHourType",
                "Tier", "IntellectualProp",
                "SimpleStatus", "ParkName",
                "DayOfWeek", "Weekend", "TimeSinceOpen", "MinutesSinceOpen",
                "CharacterExperience", "TimeSinceMidday",
                "inEMH", "EMHDay"]
newModel = ml.buildModel(RideWaits, keyFeatures, "Wait")

Fold 1
RMSE: 4.45981195015
Correlation: (0.98864328039161553, 0.0)
Accuracy: 0.945079866122
Mean Error: 2.93273955249
Median Error: 1.86152667549
-------------------------
Fold 2
RMSE: 7.09168452936
Correlation: (0.97315363695652857, 0.0)
Accuracy: 0.937891873096
Mean Error: 3.82413128425
Median Error: 1.84633818342
-------------------------
Fold 3
RMSE: 14.435777466
Correlation: (0.8700198476133294, 0.0)
Accuracy: 0.751906006995
Mean Error: 9.26637244547
Median Error: 5.68548280423
-------------------------
Fold 4
RMSE: 16.4181279374
Correlation: (0.82991709612269415, 0.0)
Accuracy: 0.759784351451
Mean Error: 10.3992049857
Median Error: 5.73389049993
-------------------------
Fold 5
RMSE: 15.1492903815
Correlation: (0.83983248561267143, 0.0)
Accuracy: 0.725774154408
Mean Error: 10.0935850863
Median Error: 6.63392802028
-------------------------
Fold 6
RMSE: 17.4433734569
Correlation: (0.82264907587775049, 0.0)
Accuracy: 0.761851206223
Mean Error: 10.9236293848
Median Error: 6.09431146

TypeError: Singleton array array('Wait',
      dtype='<U4') cannot be considered a valid collection.

In [5]:
RideWaits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69776 entries, 0 to 72206
Data columns (total 31 columns):
RideId                 69776 non-null int8
Date                   69776 non-null datetime64[ns]
Time                   69776 non-null object
Wait                   69776 non-null int64
Name                   69776 non-null int8
OpeningDate            69776 non-null datetime64[ns]
Tier                   69776 non-null int8
Location               69776 non-null object
IntellectualProp       69776 non-null int8
Status                 69776 non-null int8
Temperature            69776 non-null float64
CloudCover             69776 non-null float64
SimpleStatus           69776 non-null int8
RainAccumulation       69776 non-null float64
ParkId                 69776 non-null int8
ParkOpen               69776 non-null object
ParkClose              69776 non-null object
EMHOpen                24904 non-null object
EMHClose               24904 non-null object
ParkName               69776 non